In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import threading
import gc
import copy

sys.path.append("/end/home/rh2310/morpho_repo")
from turing.utils import *
import turing.pinns as tu
import turing.pinns_multi_nodes as tu_multi
from turing.loss_functions import *

GPU = False
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"

2022-07-06 17:15:43.037037: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1892] Ignoring visible gpu device (device: 0, name: Quadro K2000, pci bus id: 0000:05:00.0, compute capability: 3.0) with Cuda compute capability 3.0. The minimum required Cuda capability is 3.5.


In [2]:
class ASDM_multi(ASDM):
    def __init__(self,
                 dtype,
                 init_value = 10.0,
                 D_a = None,
                 D_s = None,
                 sigma_a = None,
                 sigma_s = None,
                 mu_a = None,
                 rho_a = None,
                 rho_s = None,
                 kappa_a = None,
                 print_precision=".5f"
                ):
        super().__init__(dtype, init_value, D_a, D_s, sigma_a, sigma_s, 
                         mu_a, rho_a, rho_s, kappa_a, print_precision)
    @tf.function    
    def pde(self, pinn, x):
        outputs, f_a, f_s = super().pde(pinn, x)
        return outputs, tf.concat([tf.expand_dims(f_a, axis=1), 
                          tf.expand_dims(f_s, axis=1)], axis = 1)

# Load data

In [3]:
data_path = os.path.abspath("turing.npy")
with open(data_path, 'rb') as f:
    data = np.load(f)
    
data_path = os.path.abspath("turing_t.npy")
with open(data_path, 'rb') as f:
    t_star = np.load(f) 
    
T = t_star.shape[0]    
    
L = 50
x_size = data.shape[1]
y_size = data.shape[2]
N = x_size*y_size

model_params = {'training_data_size': T*N,#T*32,
                'pde_data_size': (T*N)//(32),
                'boundary_data_size':((x_size + y_size)*T)//(8)}

dataset = create_dataset(data, t_star, N, T, L, **model_params)
lb = dataset['lb']
ub = dataset['ub']
obs_X = dataset['obs_input']
obs_Y = dataset['obs_output']
pde_X = dataset['pde']


# Test

In [4]:
layers = [3, 64, 64, 64, 64, 2]

pinn = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss = ASDM_multi(dtype=tf.float64, D_a = 0.005, D_s = 0.2)

In [5]:
model = tu_multi.TINN_multi_nodes(pinn, 
                                  pde_loss,
                                  nodes_n = 2,
                                  node_names= ["u", "v"],
                                  alpha = 0.25)

In [6]:
results = model.train(epochs = 500,
                      batch_size = 512,
                      X = obs_X,
                      Y = obs_Y,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 8.24665
total loss: 36901.80392, total regularisd loss: 36901.80392
obs u loss: 583.56867, pde u loss: 979.66713
lambda obs u: 1.00000, lambda pde u: 1.00000
obs v loss: 16305.55737, pde v loss: 19033.01075
lambda obs v: 1.00000, lambda pde v: 1.00000
sigma_a:0:10.02798, sigma_s:0:9.20963, mu_a:0:9.85684, rho_a:0:10.55152, rho_s:0:11.05547, kappa_a:0:8.90927
Time taken: 42.37s

Start of epoch 1
Training observations acc over epoch: 6.36216
total loss: 20312.88319, total regularisd loss: 20312.88319
obs u loss: 648.02180, pde u loss: 1.55567
lambda obs u: 0.31195, lambda pde u: 0.25000
obs v loss: 12381.69397, pde v loss: 7281.61175
lambda obs v: 2.47256, lambda pde v: 0.96549
sigma_a:0:10.07481, sigma_s:0:8.43107, mu_a:0:9.70449, rho_a:0:10.99664, rho_s:0:11.87508, kappa_a:0:7.99093
Time taken: 31.17s

Start of epoch 2
Training observations acc over epoch: 2.08168
total loss: 13723.22462, total regularisd loss: 18275.13640
obs u l

Training observations acc over epoch: 0.13252
total loss: 22935.60449, total regularisd loss: 978.24134
obs u loss: 247.85661, pde u loss: 51.33103
lambda obs u: 3.94069, lambda pde u: 0.00434
obs v loss: 23.55433, pde v loss: 14.50115
lambda obs v: 0.05385, lambda pde v: 0.00111
sigma_a:0:8.04979, sigma_s:0:4.33073, mu_a:0:10.86707, rho_a:0:10.03475, rho_s:0:15.37159, kappa_a:0:3.51150
Time taken: 25.82s

Start of epoch 21
Training observations acc over epoch: 0.13256
total loss: 25752.52506, total regularisd loss: 978.23050
obs u loss: 247.85711, pde u loss: 48.97580
lambda obs u: 3.94281, lambda pde u: 0.00416
obs v loss: 23.63260, pde v loss: 15.56295
lambda obs v: 0.05190, lambda pde v: 0.00112
sigma_a:0:8.01463, sigma_s:0:4.33809, mu_a:0:10.84376, rho_a:0:10.05874, rho_s:0:15.35966, kappa_a:0:3.49619
Time taken: 25.78s

Start of epoch 22
Training observations acc over epoch: 0.13254
total loss: 26361.61401, total regularisd loss: 978.54833
obs u loss: 247.81732, pde u loss: 49.68

Training observations acc over epoch: 0.13240
total loss: 26430.97725, total regularisd loss: 976.86850
obs u loss: 247.83450, pde u loss: 41.99728
lambda obs u: 3.93181, lambda pde u: 0.00372
obs v loss: 23.31726, pde v loss: 13.90712
lambda obs v: 0.06352, lambda pde v: 0.00095
sigma_a:0:7.17768, sigma_s:0:4.57222, mu_a:0:10.49081, rho_a:0:10.36027, rho_s:0:14.26607, kappa_a:0:2.96719
Time taken: 25.94s

Start of epoch 41
Training observations acc over epoch: 0.13235
total loss: 24005.29518, total regularisd loss: 975.87138
obs u loss: 247.78328, pde u loss: 38.58252
lambda obs u: 3.92892, lambda pde u: 0.00370
obs v loss: 23.27399, pde v loss: 12.51443
lambda obs v: 0.06650, lambda pde v: 0.00088
sigma_a:0:7.13985, sigma_s:0:4.59556, mu_a:0:10.46166, rho_a:0:10.38328, rho_s:0:14.18992, kappa_a:0:2.92683
Time taken: 25.82s

Start of epoch 42
Training observations acc over epoch: 0.13235
total loss: 26362.57176, total regularisd loss: 975.19939
obs u loss: 247.77533, pde u loss: 40.96

Training observations acc over epoch: 0.13229
total loss: 28905.81172, total regularisd loss: 974.30266
obs u loss: 247.76773, pde u loss: 34.46392
lambda obs u: 3.93078, lambda pde u: 0.00308
obs v loss: 23.15950, pde v loss: 15.11957
lambda obs v: 0.06522, lambda pde v: 0.00092
sigma_a:0:6.15025, sigma_s:0:4.85576, mu_a:0:10.12926, rho_a:0:10.56831, rho_s:0:12.87092, kappa_a:0:2.36605
Time taken: 26.21s

Start of epoch 61
Training observations acc over epoch: 0.13225
total loss: 25274.80909, total regularisd loss: 975.50881
obs u loss: 247.76046, pde u loss: 32.11516
lambda obs u: 3.92276, lambda pde u: 0.00321
obs v loss: 23.08331, pde v loss: 13.27004
lambda obs v: 0.07312, lambda pde v: 0.00092
sigma_a:0:6.10657, sigma_s:0:4.87587, mu_a:0:10.10629, rho_a:0:10.58245, rho_s:0:12.79257, kappa_a:0:2.33293
Time taken: 26.06s

Start of epoch 62
Training observations acc over epoch: 0.13224
total loss: 25552.04552, total regularisd loss: 973.66648
obs u loss: 247.74875, pde u loss: 33.63

Training observations acc over epoch: 0.13219
total loss: 26962.30335, total regularisd loss: 976.77383
obs u loss: 247.73145, pde u loss: 25.76422
lambda obs u: 3.93123, lambda pde u: 0.00253
obs v loss: 22.98634, pde v loss: 15.67529
lambda obs v: 0.06523, lambda pde v: 0.00100
sigma_a:0:5.04807, sigma_s:0:5.09405, mu_a:0:9.79839, rho_a:0:10.65646, rho_s:0:11.44695, kappa_a:0:1.85587
Time taken: 26.05s

Start of epoch 81
Training observations acc over epoch: 0.13217
total loss: 26341.52892, total regularisd loss: 975.44515
obs u loss: 247.72476, pde u loss: 26.91702
lambda obs u: 3.92591, lambda pde u: 0.00252
obs v loss: 22.95859, pde v loss: 15.30576
lambda obs v: 0.07055, lambda pde v: 0.00102
sigma_a:0:5.01608, sigma_s:0:5.12427, mu_a:0:9.75989, rho_a:0:10.68231, rho_s:0:11.35797, kappa_a:0:1.82033
Time taken: 26.38s

Start of epoch 82
Training observations acc over epoch: 0.13214
total loss: 24505.04688, total regularisd loss: 974.20725
obs u loss: 247.71567, pde u loss: 26.9008

...

Training observations acc over epoch: 0.00113
total loss: 4.09267, total regularisd loss: 3.48662
obs u loss: 1.39774, pde u loss: 0.95158
lambda obs u: 1.07643, lambda pde u: 0.73923
obs v loss: 0.90728, pde v loss: 0.27515
lambda obs v: 1.27739, lambda pde v: 0.90695
sigma_a:0:0.45797, sigma_s:0:0.01949, mu_a:0:2.36561, rho_a:0:1.99578, rho_s:0:0.02053, kappa_a:0:0.28437
Time taken: 27.64s

Start of epoch 401
Training observations acc over epoch: 0.00074
total loss: 3.76305, total regularisd loss: 3.19870
obs u loss: 0.77868, pde u loss: 1.62756
lambda obs u: 0.57815, lambda pde u: 1.48912
obs v loss: 0.73896, pde v loss: 0.23532
lambda obs v: 1.12532, lambda pde v: 0.80742
sigma_a:0:0.42614, sigma_s:0:0.01949, mu_a:0:2.29023, rho_a:0:1.94291, rho_s:0:0.01994, kappa_a:0:0.28303
Time taken: 27.37s

Start of epoch 402
Training observations acc over epoch: 0.00109
total loss: 4.05254, total regularisd loss: 3.28861
obs u loss: 1.36450, pde u loss: 0.87569
lambda obs u: 1.06435, lambda p

Training observations acc over epoch: 0.00026
total loss: 0.75070, total regularisd loss: 0.62226
obs u loss: 0.37334, pde u loss: 0.02440
lambda obs u: 1.13591, lambda pde u: 0.31432
obs v loss: 0.15296, pde v loss: 0.09447
lambda obs v: 0.58689, lambda pde v: 1.96289
sigma_a:0:0.01879, sigma_s:0:0.01763, mu_a:0:0.11360, rho_a:0:0.09629, rho_s:0:0.01786, kappa_a:0:0.26469
Time taken: 27.78s

Start of epoch 421
Training observations acc over epoch: 0.00025
total loss: 0.65662, total regularisd loss: 0.68874
obs u loss: 0.36961, pde u loss: 0.01042
lambda obs u: 1.23539, lambda pde u: 0.12328
obs v loss: 0.14768, pde v loss: 0.09117
lambda obs v: 0.58782, lambda pde v: 2.05352
sigma_a:0:0.01201, sigma_s:0:0.01798, mu_a:0:0.07890, rho_a:0:0.06894, rho_s:0:0.01829, kappa_a:0:0.27655
Time taken: 27.32s

Start of epoch 422
Training observations acc over epoch: 0.00023
total loss: 0.59240, total regularisd loss: 0.66768
obs u loss: 0.32577, pde u loss: 0.00586
lambda obs u: 1.22436, lambda p

Training observations acc over epoch: 0.00018
total loss: 1.19090, total regularisd loss: 0.53223
obs u loss: 0.26179, pde u loss: 0.00070
lambda obs u: 1.18340, lambda pde u: 0.00094
obs v loss: 0.10467, pde v loss: 0.07352
lambda obs v: 0.47334, lambda pde v: 2.34232
sigma_a:0:0.00000, sigma_s:0:0.01738, mu_a:0:0.01003, rho_a:0:0.01047, rho_s:0:0.01778, kappa_a:0:0.29963
Time taken: 27.18s

Start of epoch 441
Training observations acc over epoch: 0.00017
total loss: 1.20057, total regularisd loss: 0.51373
obs u loss: 0.24883, pde u loss: 0.00070
lambda obs u: 1.14795, lambda pde u: 0.00098
obs v loss: 0.10485, pde v loss: 0.07242
lambda obs v: 0.50521, lambda pde v: 2.34587
sigma_a:0:0.00005, sigma_s:0:0.01727, mu_a:0:0.01002, rho_a:0:0.01063, rho_s:0:0.01777, kappa_a:0:0.30151
Time taken: 26.63s

Start of epoch 442
Training observations acc over epoch: 0.00017
total loss: 1.17385, total regularisd loss: 0.51152
obs u loss: 0.25194, pde u loss: 0.00070
lambda obs u: 1.17452, lambda p

Training observations acc over epoch: 0.00015
total loss: 1.24594, total regularisd loss: 0.45391
obs u loss: 0.22199, pde u loss: 0.00069
lambda obs u: 1.13342, lambda pde u: 0.00100
obs v loss: 0.09300, pde v loss: 0.06747
lambda obs v: 0.50569, lambda pde v: 2.35989
sigma_a:0:0.00000, sigma_s:0:0.01690, mu_a:0:0.01007, rho_a:0:0.01034, rho_s:0:0.01784, kappa_a:0:0.28739
Time taken: 27.06s

Start of epoch 461
Training observations acc over epoch: 0.00015
total loss: 1.10693, total regularisd loss: 0.45248
obs u loss: 0.21757, pde u loss: 0.00071
lambda obs u: 1.07937, lambda pde u: 0.00086
obs v loss: 0.08814, pde v loss: 0.06835
lambda obs v: 0.42421, lambda pde v: 2.49556
sigma_a:0:0.00020, sigma_s:0:0.01747, mu_a:0:0.01004, rho_a:0:0.01042, rho_s:0:0.01746, kappa_a:0:0.28530
Time taken: 26.79s

Start of epoch 462
Training observations acc over epoch: 0.00015
total loss: 1.26118, total regularisd loss: 0.43664
obs u loss: 0.21339, pde u loss: 0.00070
lambda obs u: 1.07024, lambda p

Training observations acc over epoch: 0.00014
total loss: 1.14734, total regularisd loss: 0.40139
obs u loss: 0.19965, pde u loss: 0.00070
lambda obs u: 1.02774, lambda pde u: 0.00091
obs v loss: 0.08686, pde v loss: 0.06477
lambda obs v: 0.46116, lambda pde v: 2.51019
sigma_a:0:0.00014, sigma_s:0:0.01726, mu_a:0:0.01027, rho_a:0:0.01050, rho_s:0:0.01745, kappa_a:0:0.27890
Time taken: 26.87s

Start of epoch 481
Training observations acc over epoch: 0.00014
total loss: 1.17371, total regularisd loss: 0.40892
obs u loss: 0.20118, pde u loss: 0.00070
lambda obs u: 1.04432, lambda pde u: 0.00091
obs v loss: 0.08503, pde v loss: 0.06491
lambda obs v: 0.44679, lambda pde v: 2.50798
sigma_a:0:0.00003, sigma_s:0:0.01724, mu_a:0:0.01028, rho_a:0:0.01044, rho_s:0:0.01759, kappa_a:0:0.27915
Time taken: 26.82s

Start of epoch 482
Training observations acc over epoch: 0.00014
total loss: 1.16542, total regularisd loss: 0.40299
obs u loss: 0.19415, pde u loss: 0.00070
lambda obs u: 1.00842, lambda p